<a href="https://colab.research.google.com/github/Palak2506/julep-ai/blob/main/juleptask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install julep


In [17]:
import os
import requests # For making HTTP requests to external APIs
import json
import time # For adding delays to avoid rate limits

# --- Step 0.1: Install Julep & requests (Run this first in a Colab cell) ---
# !pip install julep requests

# --- Step 0.2: Securely load API Keys from Colab Secrets ---
# Go to the left sidebar -> Secrets (key icon) -> Add new secret.
# Name: JULEP_API_KEY, Value: Your_Actual_Julep_API_Key
# Name: OPENWEATHERMAP_API_KEY, Value: Your_Actual_OpenWeatherMap_API_Key
# Make sure "Notebook access" is ON for both.

try:
    from google.colab import userdata
    try:
        os.environ["JULEP_API_KEY"] = userdata.get('JULEP_API_KEY')
        os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get('OPENWEATHERMAP_API_KEY')
        print("API Keys loaded from Colab Secrets.")
    except userdata.SecretNotFoundError as e:
        print(f"Error loading secret from Colab: {e}. Please ensure the secret exists and Notebook access is ON.")
        print("Falling back to checking environment variables.")
        # Continue to the environment variable check later in the workflow function
except ImportError:
    print("Running outside Colab or 'google.colab' not found. "
          "Ensure JULEP_API_KEY and OPENWEATHERMAP_API_KEY are set as environment variables.")
    # Fallback for local development or if not using Colab secrets directly
    # os.environ["JULEP_API_KEY"] = "YOUR_JULEP_API_KEY_HERE"
    # os.environ["OPENWEATHERMAP_API_KEY"] = "YOUR_OPENWEATHERMAP_API_KEY_HERE"

# --- Step 1: Initialize Julep Client ---
from julep import Julep
# Check if JULEP_API_KEY is set before initializing
julep_api_key = os.environ.get("JULEP_API_KEY")
if not julep_api_key:
     print("WARNING: JULEP_API_KEY is not set. Julep client may not initialize correctly or API calls will fail.")
     # Attempt to initialize client anyway, but user should be aware it might fail
     client = Julep()
else:
    client = Julep() # Julep client will pick up the key from os.environ
    print("Julep client initialized successfully.")



# --- Helper Function: Get Real-Time Weather ---
def get_current_weather(city_name: str, api_key: str) -> dict:
    """
    Fetches real-time weather data for a given city using OpenWeatherMap API.
    Returns a dictionary with weather description, temperature, and basic conditions.
    """
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    # Using 'q=' for city name directly, 'units=metric' for Celsius
    full_url = f"{base_url}q={city_name}&units=metric&appid={api_key}"
    try:
        response = requests.get(full_url)
        response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        if data.get("cod") == 200: # Check if request was successful
            weather_desc = data["weather"][0]["description"].capitalize()
            temp = data["main"]["temp"]
            conditions = data["weather"][0]["main"] # e.g., 'Clouds', 'Rain', 'Clear'

            return {
                "description": weather_desc,
                "temperature": temp,
                "main_conditions": conditions,
                "success": True
            }
        else:
            print(f"  Weather API error for {city_name}: {data.get('message', 'Unknown error')}")
            return {"success": False, "error": data.get('message', 'Unknown error')}
    except requests.exceptions.RequestException as e:
        print(f"  Network error fetching weather for {city_name}: {e}")
        return {"success": False, "error": str(e)}
    except Exception as e:
        print(f"  An unexpected error occurred getting weather for {city_name}: {e}")
        return {"success": False, "error": str(e)}

# --- Helper Function: Simulate Restaurant Search (No Real API Key Needed) ---
# In a real scenario, this would call Google Places, Yelp, etc.
def find_top_restaurants_for_dish(city: str, dish: str, dining_type: str) -> str:
    """
    Simulates finding top-rated restaurants for a dish.
    In a real application, this would query a restaurant API.
    """
    # Simple rule-based simulation for demonstration
    if "Indoor" in dining_type:
        type_hint = "cozy indoor spot"
    elif "Outdoor" in dining_type:
        type_hint = "lovely outdoor patio"
    else:
        type_hint = "versatile dining option"

    # Provide more specific (but still simulated) restaurant names for some cities/dishes
    simulated_restaurants = {
        "New York City": {
            "New York Style Pizza": "Joe's Pizza (Casual, Dining: Both)",
            "Bagel with Lox": "Russ & Daughters Cafe (Deli/Cafe, Dining: Indoor)",
            "Cheesecake": "Junior's Restaurant (Classic Diner, Dining: Indoor)",
        },
        "Paris": {
            "Croissant": "Du Pain et des Idées (Artisan Bakery, Dining: Indoor/Takeaway)",
            "Escargots": "Le Comptoir du Relais (Bistro, Dining: Indoor)",
            "Macarons": "Pierre Hermé (Patisserie, Dining: Takeaway)",
        },
        "Tokyo": {
            "Sushi": "Sukiyabashi Jiro (Omakase Sushi, Dining: Indoor)",
            "Ramen": "Ichiran Ramen (Counter Service, Dining: Indoor)",
            "Tempura": "Tempura Kondo (Fine Dining, Dining: Indoor)",
        },
        "Rome": {
            "Cacio e Pepe": "Trattoria Da Enzo al 29 (Traditional Trattoria, Dining: Both)",
            "Supplí": "Supplizio (Street Food, Dining: Takeaway/Limited Indoor)",
            "Gelato": "Giolitti (Historic Gelateria, Dining: Indoor/Takeaway)",
        }
    }

    city_restaurants = simulated_restaurants.get(city, {})
    restaurant_info = city_restaurants.get(dish)

    if restaurant_info:
        return f"- {dish}: {restaurant_info}"
    else:
        # Fallback if dish not in our specific simulated list
        return f"- {dish}: [Top-Rated Restaurant in {city}] (A {type_hint} for {dish}, Dining: {dining_type})"


# --- Step 2: Define the Core Workflow Function ---
def create_foodie_tour_workflow(city_name: str):
    """
    Creates a one-day foodie tour for a given city, integrating real-time weather
    and simulated restaurant data, powered by Julep's LLM.
    """
    print(f"\n--- Generating foodie tour for: {city_name} ---")

    # --- Step 2.1: Get Real-Time Weather & Determine Dining Suggestion ---
    weather_api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    if not weather_api_key:
        print("  WARNING: OPENWEATHERMAP_API_KEY not set. Cannot fetch real weather.")
        # Fallback to LLM simulation if API key is missing
        weather_info_llm_prompt = f"""
        What is today's general weather outlook for {city_name}?
        Based on the weather, should dining be suggested primarily indoor or outdoor?
        Be concise with the weather, e.g., "Sunny with a light breeze".
        Format your response clearly:
        Weather: [Description of weather]
        Dining Suggestion: [Indoor/Outdoor/Both]
        """
        print(f"  Simulating weather and dining suggestion for {city_name} via LLM...")
        weather_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": weather_info_llm_prompt}]
        )
        weather_output_raw = weather_response.choices[0].message.content.strip()
        weather_description_for_narrative = weather_output_raw.split('Weather: ')[1].split('Dining Suggestion: ')[0].strip()
        dining_suggestion = "Both" # Default if parsing fails
        if "Dining Suggestion: Indoor" in weather_output_raw:
            dining_suggestion = "Indoor"
        elif "Dining Suggestion: Outdoor" in weather_output_raw:
            dining_suggestion = "Outdoor"
        print(f"  Weather (LLM simulated): {weather_description_for_narrative}, Dining: {dining_suggestion}")

    else:
        print(f"  Fetching real-time weather for {city_name}...")
        weather_data = get_current_weather(city_name, weather_api_key)
        if weather_data["success"]:
            weather_description_for_narrative = (
                f"{weather_data['description']} with a temperature of {weather_data['temperature']}°C."
            )
            # Logic to determine dining suggestion based on real weather data
            # This could be more complex (e.g., rain, very cold/hot)
            if "rain" in weather_data['description'].lower() or weather_data['temperature'] < 5:
                dining_suggestion = "Indoor"
            elif "clear" in weather_data['description'].lower() and weather_data['temperature'] > 18:
                dining_suggestion = "Outdoor"
            else:
                dining_suggestion = "Both"
            print(f"  Weather (Real-time): {weather_description_for_narrative}, Dining: {dining_suggestion}")
        else:
            print(f"  Failed to get real weather: {weather_data['error']}. Falling back to LLM simulation for dining suggestion.")
            # If real weather fails, ask LLM for dining suggestion anyway
            weather_info_llm_prompt = f"""
            What is a typical dining suggestion (Indoor/Outdoor/Both) for {city_name} given generally pleasant weather?
            Format: Dining Suggestion: [Indoor/Outdoor/Both]
            """
            dining_response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": weather_info_llm_prompt}]
            )
            dining_suggestion = dining_response.choices[0].message.content.strip().replace("Dining Suggestion: ", "")
            weather_description_for_narrative = f"Typical weather for {city_name}." # Default if real fails


    # --- Step 2.2: Pick 3 Iconic Local Dishes (LLM Assisted) ---
    dishes_prompt = f"""
    List 3 iconic, must-try local dishes from {city_name}.
    Provide only the dish names, separated by commas.
    Example: Pizza Margherita, Pasta Carbonara, Tiramisu
    """
    print(f"  Querying LLM for iconic dishes for {city_name}...")
    dishes_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": dishes_prompt}]
    )
    iconic_dishes_str = dishes_response.choices[0].message.content.strip()
    iconic_dishes = [dish.strip() for dish in iconic_dishes_str.split(',') if dish.strip()]
    print(f"  Dishes suggested: {', '.join(iconic_dishes)}")
    time.sleep(1) # Pause to avoid hitting LLM rate limits


    # --- Step 2.3: Find Top-Rated Restaurants for these dishes (Simulated or Real) ---
    # This section demonstrates how you would integrate a restaurant API.
    # For now, it uses the `find_top_restaurants_for_dish` helper which simulates.
    restaurants_info_lines = []
    print(f"  Finding restaurants for dishes in {city_name}...")
    for dish in iconic_dishes:
        # In a real app:
        # restaurant_data_from_api = call_real_restaurant_api(city_name, dish, dining_suggestion)
        # formatted_restaurant_info = format_api_response(restaurant_data_from_api)
        # restaurants_info_lines.append(formatted_restaurant_info)

        # For this example, we use the simulated function:
        simulated_info = find_top_restaurants_for_dish(city_name, dish, dining_suggestion)
        restaurants_info_lines.append(simulated_info)
        time.sleep(0.5) # Small pause for simulated calls

    restaurants_info_formatted = "\n".join(restaurants_info_lines)
    print(f"  Restaurant info collected:\n{restaurants_info_formatted}")


    # --- Step 2.4: Create a Delightful One-Day Foodie Tour Narrative (LLM) ---
    narrative_prompt = f"""
    Craft a delightful and engaging one-day "foodie tour" narrative for a visitor in {city_name}.
    The narrative should cover breakfast, lunch, and dinner, factoring in the current weather and dining suggestions.

    Here's the information to use:
    - Today's weather: {weather_description_for_narrative}
    - Overall dining preference: {dining_suggestion}
    - Iconic dishes and their suggested restaurants:
      {restaurants_info_formatted}

    Structure your narrative with clear sections for:
    **Morning: Breakfast & Start of the Day**
    **Midday: Lunch & Culinary Exploration**
    **Evening: Dinner & Grand Finale**

    Make it sound like a personal recommendation, highlight the dishes and restaurants,
    and subtly weave in how the weather might influence the experience. Keep it concise, engaging, and around 250-350 words.
    """
    print(f"  Generating foodie tour narrative for {city_name}...")
    tour_narrative_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": narrative_prompt}]
    )
    foodie_tour_narrative = tour_narrative_response.choices[0].message.content.strip()
    print(f"  Narrative generated.")
    time.sleep(1) # Pause to avoid hitting LLM rate limits


    return {
        "city": city_name,
        "weather_description": weather_description_for_narrative,
        "dining_suggestion": dining_suggestion,
        "iconic_dishes": iconic_dishes,
        "restaurants_info": restaurants_info_formatted,
        "foodie_tour_narrative": foodie_tour_narrative
    }

# --- Step 3: Execute the Workflow for Multiple Cities ---
if __name__ == "__main__":
    cities_to_tour = ["London", "Kyoto", "Barcelona"] # Changed some cities for variety

    all_tour_details = []

    for city in cities_to_tour:
        try:
            tour_details = create_foodie_tour_workflow(city)
            all_tour_details.append(tour_details)
            print(f"\n--- Successfully generated tour for {city} ---\n")
            print("="*80)
        except Exception as e:
            print(f"\n!!! Error generating tour for {city}: {e} !!!\n")
            print("="*80)

    print("\n\n--- All Foodie Tours Generation Complete ---")

    # --- Step 4 (Optional): Display or Save Results ---
    print("\n\n--- Summary of Generated Foodie Tour Narratives ---")
    for tour in all_tour_details:
        print(f"\n### Foodie Tour for {tour['city']} ###")
        print(f"Weather: {tour['weather_description']}, Dining: {tour['dining_suggestion']}")
        print(f"Dishes: {', '.join(tour['iconic_dishes'])}")
        print("Restaurants:\n" + tour['restaurants_info'])
        print("\nNarrative:\n" + tour['foodie_tour_narrative'])
        print("\n--------------------------------------------------")

    output_filename = "foodie_tours_realtime_summary.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(all_tour_details, f, indent=4, ensure_ascii=False)
    print(f"\nAll tour details saved to {output_filename}")

Error loading secret from Colab: Secret OPENWEATHERMAP_API_KEY does not exist.. Please ensure the secret exists and Notebook access is ON.
Falling back to checking environment variables.
Julep client initialized successfully.

--- Generating foodie tour for: London ---
  Simulating weather and dining suggestion for London via LLM...

!!! Error generating tour for London: 'Julep' object has no attribute 'chat' !!!


--- Generating foodie tour for: Kyoto ---
  Simulating weather and dining suggestion for Kyoto via LLM...

!!! Error generating tour for Kyoto: 'Julep' object has no attribute 'chat' !!!


--- Generating foodie tour for: Barcelona ---
  Simulating weather and dining suggestion for Barcelona via LLM...

!!! Error generating tour for Barcelona: 'Julep' object has no attribute 'chat' !!!



--- All Foodie Tours Generation Complete ---


--- Summary of Generated Foodie Tour Narratives ---

All tour details saved to foodie_tours_realtime_summary.json
